In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
from config import db_password
import os
from os import listdir
from os.path import isfile, join
from pathlib import Path

In [39]:
# Make dataframes for psy2019 and psy2020 CSVs
psy2019_df = pd.read_csv(r'resources\psy2019.csv')
psy2020_df = pd.read_csv(r'resources\psy2020.csv')


In [40]:
# Drop rows with bugged values
psy2019_df= psy2019_df.drop(index=[52,53,54,55,56,57,58])

In [41]:
# Drop rows with bugged values
psy2020_df = psy2020_df.drop(index=[52,53,54,55,56,57])

In [42]:
# Drop unnecessary data from Area Name column 
psy2019_df['Area Name'] = psy2019_df['Area Name'].str.replace(r"\(.*\)","")
psy2020_df['Area Name'] = psy2020_df['Area Name'].str.replace(r"\(.*\)","")


C:\Users\noaht\AppData\Local\Temp/ipykernel_11896/2461758949.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  psy2019_df['Area Name'] = psy2019_df['Area Name'].str.replace(r"\(.*\)","")
C:\Users\noaht\AppData\Local\Temp/ipykernel_11896/2461758949.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  psy2020_df['Area Name'] = psy2020_df['Area Name'].str.replace(r"\(.*\)","")


In [43]:
# Rename all column names to be more readable and workable for postgres 
psy2020_df = psy2020_df.rename(columns={'Area Name' : 'State', 'Employment(1)' : 'psy2020_number_employed', 'Annual mean wage(2)' : 'psy2020_annual_mean_wage', 'Employment per 1,000 jobs' : 'psy2020_employ_per_1k_jobs' , 'Location Quotient' : 'psy2020_loc_quotient' })

In [44]:
# Rename all column names to be more readable and workable for postgres 
psy2019_df = psy2019_df.rename(columns={'Area Name' : 'State', 'Employment(1)' : 'psy2019_number_employed', 'Employment percent relative standard error(3)' : 'psy2019_percent_relative_standard_error', 'Annual mean wage(2)' : 'psy2019_annual_mean_wage', 'Employment per 1,000 jobs' : 'psy2019_employ_per_1k_jobs' , 'Location Quotient' : 'psy2019_loc_quotient' })

In [45]:
# Make psyjobs dataframe 
psyjobs_df = pd.read_csv(r'resources\us_psyjobs_trends.csv')

In [50]:
psyjobs_df.head()

,STATE,TOT_EMP,JOBS_1000,LOC_Q,Year
0,Alabama,"1,750",0.901,2.29,2018
1,Alaska,160,0.496,1.26,2018
2,Arkansas,340,0.279,0.71,2018
3,California,"1,910",0.112,0.29,2018
4,Colorado,520,0.198,0.50,2018


In [59]:
# Split up psyjobs df by year
psyjobs_df2018 = psyjobs_df.groupby(psyjobs_df.Year).get_group(2018)
psyjobs_df2019 = psyjobs_df.groupby(psyjobs_df.Year).get_group(2019)
psyjobs_df2020 = psyjobs_df.groupby(psyjobs_df.Year).get_group(2020)


In [60]:
# Drop year column for psyjobs dataframes
psyjobs_df2018 = psyjobs_df2018.drop(columns=['Year'])
psyjobs_df2019 = psyjobs_df2019.drop(columns=['Year'])
psyjobs_df2020 = psyjobs_df2020.drop(columns=['Year'])


In [61]:
# Add suffix for variable names
psyjobs_df2018 = psyjobs_df2018.add_suffix('_2018')
psyjobs_df2019 = psyjobs_df2019.add_suffix('_2019')
psyjobs_df2020 = psyjobs_df2020.add_suffix('_2020')

In [62]:
# Standardize state column
psyjobs_df2018 = psyjobs_df2018.rename(columns={'STATE_2018' : 'State'})

In [63]:
# Standardize state column
psyjobs_df2019 = psyjobs_df2019.rename(columns={'STATE_2019' : 'State'})

In [64]:
# Standardize state column
psyjobs_df2020 = psyjobs_df2020.rename(columns={'STATE_2020' : 'State'})

In [65]:
# Setup postgres variables
pg_user = "postgres"
pg_pwd = db_password
pg_port = "5432"
host = 'project-vu-database-piecharts.c7rvpt2rehpr.us-east-2.rds.amazonaws.com'

In [66]:
# Create connection string
db_string = "postgresql://{username}:{password}@{host}:{port}/project_db".format(username=pg_user, password=pg_pwd, host = host, port=pg_port)


In [67]:
# Connect to the database engine
engine = create_engine(db_string)

In [68]:
# Load psyjobs dataframes to sql server
psyjobs_df2020.to_sql(name='psyjobs2020', if_exists='replace', con=engine, index=False)
psyjobs_df2019.to_sql(name='psyjobs2019', if_exists='replace', con=engine, index=False)
psyjobs_df2018.to_sql(name='psyjobs2018', if_exists='replace', con=engine, index=False)


In [69]:
# Load general psy dataframes to sql server
psy2019_df.to_sql(name='psy2019', if_exists='replace', con=engine, index=False)
psy2020_df.to_sql(name='psy2020', if_exists='replace', con=engine, index=False)